In [44]:
import torch
import torch.nn.functional as F
import ast
import pandas as pd
from sklearn.model_selection import train_test_split

In [86]:
df1 = pd.read_csv("/Users/pranavtyagi/Projects/masterthesis/data/predictions/feb/analysis_metrics.csv")

In [87]:
train_data, temp_data = train_test_split(df1, test_size=0.3, random_state=42)
val_data, df = train_test_split(temp_data, test_size=1/3, random_state=42)

In [88]:
df.columns

Index(['conversation_id', 'tweet_text', 'roberta-large-predictions',
       'mapped_predictions', 'averaged_predictions',
       'cosine_similarity_tweet_replies', 'clip_predictions',
       'cosine_similarity_clip_replies', 'cosine_similarity_clip_tweet',
       'matched_filename', 'tweet_distribution', 'average_reply_distribution',
       'clip_distribution', 'clip_tweet_dist', 'cosine_similarity_multi',
       'jsd', 'jsd_image', 'jsd_multi', 'jsd_image_reply', 'tweet_entropy',
       'reply_entropy', 'multi_entropy'],
      dtype='object')

In [90]:
def parse_line(predictions_str, labels_str):
    try:
        predictions_dict = ast.literal_eval(predictions_str)
        labels_dict = ast.literal_eval(labels_str)
        
        # Convert to tensors
        probs = torch.tensor(list(predictions_dict.values()), dtype=torch.float32)
        labels = torch.tensor(list(labels_dict.values()), dtype=torch.float32)
        return probs, labels
    except (SyntaxError, ValueError):
        return None, None

prediction_columns = ['tweet_distribution', 'clip_distribution', 'clip_tweet_dist'] 
kl_divergences_dict = {}
mean_squared_errors_dict = {}

for pred_col in prediction_columns:
    kl_divergences = []
    mean_squared_errors = []
    
    for _, row in df.iterrows():
        probs, labels = parse_line(row[pred_col], row['average_reply_distribution'])
        if probs is None or labels is None:
            continue  # Skip invalid rows

        # Ensure the tensors are properly normalized
        probs = probs / probs.sum()  # Convert to valid probability distribution
        labels = labels / labels.sum()  # Normalize labels as well

        batch_kl_divergences = F.kl_div(probs.log(), labels, reduction='sum').item()
        batch_mse = F.mse_loss(probs, labels, reduction='mean').item()

        kl_divergences.append(batch_kl_divergences)
        mean_squared_errors.append(batch_mse)
    
    # Store results in dictionary
    kl_divergences_dict[f'KL_Divergence_{pred_col}'] = kl_divergences
    mean_squared_errors_dict[f'MSE_{pred_col}'] = mean_squared_errors

# Ensure DataFrame dimensions match
df = df.iloc[:len(kl_divergences)]  

# Add results to DataFrame
for key, values in kl_divergences_dict.items():
    df[key] = values
for key, values in mean_squared_errors_dict.items():
    df[key] = values

/var/folders/yp/dcvlqkwd0jgdyxjp_q1f8w440000gn/T/ipykernel_37125/3403478139.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key] = values
/var/folders/yp/dcvlqkwd0jgdyxjp_q1f8w440000gn/T/ipykernel_37125/3403478139.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key] = values
/var/folders/yp/dcvlqkwd0jgdyxjp_q1f8w440000gn/T/ipykernel_37125/3403478139.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

In [91]:
df

,conversation_id,tweet_text,roberta-large-predictions,mapped_predictions,averaged_predictions,cosine_similarity_tweet_replies,clip_predictions,cosine_similarity_clip_replies,cosine_similarity_clip_tweet,matched_filename,...,jsd_image_reply,tweet_entropy,reply_entropy,multi_entropy,KL_Divergence_tweet_distribution,KL_Divergence_clip_distribution,KL_Divergence_clip_tweet_dist,MSE_tweet_distribution,MSE_clip_distribution,MSE_clip_tweet_dist
198,1096448460583260161,Our students took the lead in the fight agains...,"[('optimism', 0.9810001850128174), ('joy', 0.8...","[('fear', 0.011164323351160214), ('disgust', 0...","[('fear', 0.06047392221213688), ('disgust', 0....",0.339843,"[('anger', 0.5489631), ('disgust', 0.13539463)...",0.736023,0.115306,id_1096448460583260161_2019-02-15.jpg,...,0.332901,0.894837,2.411120,2.119343,2.127488,0.687516,0.409579,0.105137,0.052507,0.027742
332,1096541711847833601,Why are Arctic fisheries under threat due to c...,"[('fear', 0.9263613820075989), ('sadness', 0.4...","[('fear', 0.48925698009041596), ('disgust', 0....","[('fear', 0.08450050080627194), ('disgust', 0....",0.542696,"[('anger', 0.31692), ('sadness', 0.2740194), (...",0.494711,0.629708,id_1096541711847833601_2019-02-15.jpg,...,0.481270,2.008410,2.096574,2.366826,0.526433,0.695098,0.560914,0.047499,0.042861,0.037811
43,1095943507779305472,Everywhere there are signs that we have reache...,"[('fear', 0.871435284614563), ('sadness', 0.61...","[('fear', 0.49675462303614526), ('disgust', 0....","[('fear', 0.0318985875978461), ('disgust', 0.0...",0.269678,"[('surprise', 0.3934697), ('disgust', 0.300639...",0.662813,0.344603,id_1095943507779305472_2019-02-14.jpg,...,0.486785,1.990189,1.887569,2.389172,1.327314,0.766100,0.974720,0.080078,0.053326,0.040766
425,1096757339749679104,Editorial: Mitch McConnell doubtlessly underst...,"[('disgust', 0.7204411625862122), ('anticipati...","[('fear', 0.13702042579834062), ('disgust', 0....","[('fear', 0.21606504528862117), ('disgust', 0....",0.887880,"[('anger', 0.65399826), ('disgust', 0.11806615...",0.325795,0.538123,id_1096757339749679104_2019-02-16.jpg,...,0.590347,2.317214,2.420347,2.240631,0.293463,0.485763,0.436029,0.007880,0.042610,0.028290
1379,1100430895083868160,This photo worries me #February #climatechange...,"[('fear', 0.9942943453788757), ('pessimism', 0...","[('fear', 0.7040371969671779), ('disgust', 0.0...","[('fear', 0.4635530298826913), ('disgust', 0.2...",0.890262,"[('anger', 0.52634525), ('sadness', 0.15412398...",0.607254,0.317794,id_1100430895083868160_2019-02-26.jpg,...,0.396548,1.433912,2.030842,2.088118,0.515442,0.445560,0.122950,0.019670,0.009042,0.006012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1539,1100873906594959360,27th February 2018 vs 27th February 2019. But ...,"[('disgust', 0.6010696291923523), ('anticipati...","[('fear', 0.041227948416333864), ('disgust', 0...","[('fear', 0.10460124212115876), ('disgust', 0....",0.743647,"[('disgust', 0.36531302), ('anger', 0.30135813...",0.660809,0.952324,id_1100873906594959360_2019-02-27.jpg,...,0.433890,2.064323,2.385512,2.111497,0.398629,0.581145,0.456085,0.022315,0.030722,0.024026
1106,1098639572316160001,#Women4Climate thanks [USER] for this event pr...,"[('optimism', 0.9892467856407166), ('joy', 0.9...","[('fear', 0.011701253456356545), ('disgust', 0...","[('fear', 0.047613872014641734), ('disgust', 0...",0.155138,"[('anger', 0.3910448), ('sadness', 0.31996816)...",0.777300,0.047288,id_1098639572316160001_2019-02-21.jpg,...,0.351054,0.806308,2.124179,2.207442,3.560697,1.384152,0.632145,0.140824,0.038910,0.041484
1420,1100522521047252993,"Arthur’s Seat 2018, vs Arthur’s Seat 2019.Clim...","[('optimism', 0.5123183727264404), ('disgust',...","[('fear', 0.16227357463072134), ('disgust', 0....","[('fear', 0.12573714559088056), ('disgust', 0....",0.961935,"[('anger', 0.5869566), ('fear', 0.11374291), (...",0.701974,0.539706,id_1100522521047252993_2019-02-26.jpg,...,0.372500,2.484331,2.475295,

In [92]:
df.to_csv("results_feb.csv", index=False)

In [93]:
df=pd.read_csv("results_aug.csv")

In [94]:
df

,conversation_id,tweet_text,roberta-large-predictions,averaged_predictions,mapped_predictions,cosine_similarity_tweet_replies,clip_predictions,cosine_similarity_clip_replies,cosine_similarity_clip_tweet,similarity_bin,...,jsd_image_reply,tweet_entropy,reply_entropy,multi_entropy,KL_Divergence_tweet_distribution,KL_Divergence_clip_distribution,KL_Divergence_clip_tweet_dist,MSE_tweet_distribution,MSE_clip_distribution,MSE_clip_tweet_dist
0,1166008123263475712,#AlTenoorOverflow #Africafires#Angola#Zambia#T...,"[('anticipation', 0.18425847589969635), ('ange...","[('anger', 0.06943728003861536), ('fear', 0.48...","[('anger', 0.21514055782994462), ('fear', 0.15...",0.549003,"[('fear', 0.3789367), ('anger', 0.26606992), (...",0.805413,0.782031,0.6-0.8,...,0.372395,2.392732,1.894418,2.397760,0.659006,0.581619,0.479665,0.043325,0.035957,0.027494
1,1158024573243334657,Sage words from co-founder of the [USER] [USER...,"[('disgust', 0.9435292482376099), ('anger', 0....","[('anger', 0.12345011625812803), ('fear', 0.11...","[('anger', 0.3335972931162361), ('fear', 0.058...",0.741571,"[('surprise', 0.3979619), ('disgust', 0.192632...",0.718972,0.667360,0.6-0.8,...,0.410953,2.157165,2.474610,2.316720,0.340141,0.938059,0.299686,0.020341,0.028038,0.013925
2,1161462789795221504,So I know they are #ClimateChangedeniers.But I...,"[('disgust', 0.9702249765396118), ('anger', 0....","[('anger', 0.2677825772720873), ('fear', 0.104...","[('anger', 0.33216690109041486), ('fear', 0.12...",0.970299,"[('surprise', 0.39039624), ('anger', 0.3012097...",0.632252,0.620269,0.6-0.8,...,0.428990,2.045863,2.225498,2.320151,0.060455,0.407387,0.160361,0.002703,0.021938,0.008429
3,1163191441306398721,Is Climate Change real? That’s not up to us to...,"[('anticipation', 0.9576886892318726), ('optim...","[('anger', 0.15045814225567147), ('fear', 0.07...","[('anger', 0.08089625640880886), ('fear', 0.07...",0.938699,"[('anger', 0.45440218), ('disgust', 0.20690694...",0.622256,0.432749,0.4-0.6,...,0.458495,1.965242,2.367059,2.300349,0.113712,0.845224,0.113358,0.007616,0.030372,0.005374
4,1164233724772462592,Why isn’t everyone doing this?[URL] [USER] #cl...,"[('optimism', 0.792866587638855), ('sadness', ...","[('anger', 0.12882927371328437), ('fear', 0.17...","[('anger', 0.0580449479990004), ('fear', 0.028...",0.838172,"[('anger', 0.5921461), ('sadness', 0.21181808)...",0.565835,0.321836,0.2-0.4,...,0.516915,2.110238,2.389405,2.282167,0.305661,1.565800,0.283622,0.014076,0.058481,0.011233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730,1164909872409972736,"The diary of young ⁦[USER]⁩, 1994: never not d...","[('sadness', 0.9923811554908752), ('pessimism'...","[('anger', 0.029483021896698425), ('fear', 0.0...","[('anger', 0.011742866453751184), ('fear', 0.4...",0.501565,"[('fear', 0.418049), ('anger', 0.24365497), ('...",0.565783,0.583955,0.4-0.6,...,0.526592,1.593220,2.256675,2.139830,1.403162,0.933948,0.691342,0.054487,0.042847,0.034676
731,1157266962671689733,If terraforming is the transformation of anoth...,"[('fear', 0.5643811225891113), ('anticipation'...","[('anger', 0.08946653180516631), ('fear', 0.15...","[('anger', 0.06552404663067052), ('fear', 0.31...",0.636690,"[('anger', 0.5833039), ('sadness', 0.21278773)...",0.325762,0.357303,0.2-0.4,...,0.630028,2.188562,2.347650,2.308614,0.857563,1.886015,1.112878,0.028917,0.066930,0.032766
732,1162437587715289089,I’m with 700+ Catholic sisters at #LCWR2019. W...,"[('optimism', 0.9784810543060303), ('sadness',...","[('anger', 0.004858315480424416), ('fear', 0.0...","[('anger', 0.06499159918398285), ('fear', 0.12...",0.932122,"[('disgust', 0.36011955), ('anger', 0.34713572...",0.084196,0.287512,0.2-0.4,...,0.879451,2.052232,0.572612,2.439128,0.391660,3.720033,0.980267,0.031450,0.179255,0.084364
733,1159941018764173317,What an adventure! Three and a half years goes...,"[('joy', 0.9972921013832092), ('optimism', 0.8...","[('anger', 0.17794349822853445), ('fear', 0.03...","[('anger', 0.

In [95]:
df.columns

Index(['conversation_id', 'tweet_text', 'roberta-large-predictions',
       'averaged_predictions', 'mapped_predictions',
       'cosine_similarity_tweet_replies', 'clip_predictions',
       'cosine_similarity_clip_replies', 'cosine_similarity_clip_tweet',
       'similarity_bin', 'matched_filename', 'tweet_distribution',
       'average_reply_distribution', 'clip_distribution', 'clip_tweet_dist',
       'cosine_similarity_multi', 'jsd', 'jsd_image', 'jsd_multi',
       'jsd_image_reply', 'tweet_entropy', 'reply_entropy', 'multi_entropy',
       'KL_Divergence_tweet_distribution', 'KL_Divergence_clip_distribution',
       'KL_Divergence_clip_tweet_dist', 'MSE_tweet_distribution',
       'MSE_clip_distribution', 'MSE_clip_tweet_dist'],
      dtype='object')

In [119]:
df1= pd.read_csv("/Users/pranavtyagi/Projects/masterthesis/data/predictions/aug/test_predictions_with_metrics_text.csv")
df2= pd.read_csv("/Users/pranavtyagi/Projects/masterthesis/data/predictions/aug/test_predictions_with_metrics_image.csv")
df3= pd.read_csv("/Users/pranavtyagi/Projects/masterthesis/data/predictions/aug/test_predictions_with_metrics_multi.csv")

In [120]:
merged_df = df1.merge(df2, on='conversation_id')

In [127]:
merged_df

,conversation_id,predictions_text_base,cosine_similarity_model_text_base,kl_divergence_text_base,mean_squared_error_text_base,predictions_image_base,cosine_similarity_model_image_base,kl_divergence_image_base,mean_squared_error_image_base,predictions_multi_base,cosine_similarity_model_multi_base,kl_divergence_multi_base,mean_squared_error_multi_base
0,1166008123263475712,"[('anger', 0.17008963227272034), ('sadness', 0...",0.617451,0.596990,0.035849,"[('anger', 0.11810187250375748), ('sadness', 0...",0.510256,0.757818,0.045332,"[('anger', 0.10888910293579102), ('sadness', 0...",0.468703,0.843957,0.049363
1,1158024573243334657,"[('anger', 0.16087716817855835), ('sadness', 0...",0.920432,0.079872,0.004975,"[('anger', 0.13490606844425201), ('sadness', 0...",0.996235,0.006962,0.000267,"[('anger', 0.1712423413991928), ('sadness', 0....",0.966738,0.039413,0.002149
2,1161462789795221504,"[('anger', 0.16136960685253143), ('sadness', 0...",0.745284,0.357470,0.018154,"[('anger', 0.13151054084300995), ('sadness', 0...",0.724121,0.373151,0.019820,"[('anger', 0.18524488806724548), ('sadness', 0...",0.825245,0.270557,0.012765
3,1163191441306398721,"[('anger', 0.17034029960632324), ('sadness', 0...",0.831767,0.204059,0.011251,"[('anger', 0.17220427095890045), ('sadness', 0...",0.936768,0.065914,0.004436,"[('anger', 0.15097835659980774), ('sadness', 0...",0.980087,0.020357,0.001476
4,1164233724772462592,"[('anger', 0.1831539273262024), ('sadness', 0....",0.886504,0.157142,0.007363,"[('anger', 0.13731467723846436), ('sadness', 0...",0.777628,0.304395,0.015191,"[('anger', 0.12437426298856735), ('sadness', 0...",0.734397,0.374825,0.018336
...,...,...,...,...,...,...,...,...,...,...,...,...,...
730,1164909872409972736,"[('anger', 0.17293518781661987), ('sadness', 0...",0.742247,0.343101,0.018812,"[('anger', 0.1053723692893982), ('sadness', 0....",0.793000,0.242755,0.016604,"[('anger', 0.10436020791530609), ('sadness', 0...",0.852124,0.169199,0.011279
731,1157266962671689733,"[('anger', 0.19197547435760498), ('sadness', 0...",0.876641,0.136145,0.008820,"[('anger', 0.12357351928949356), ('sadness', 0...",0.959709,0.053473,0.003055,"[('anger', 0.12282855808734894), ('sadness', 0...",0.932210,0.069589,0.004986
732,1162437587715289089,"[('anger', 0.15329782664775848), ('sadness', 0...",0.508829,1.256120,0.103514,"[('anger', 0.11171422898769379), ('sadness', 0...",0.818509,0.648870,0.057328,"[('anger', 0.11640714108943939), ('sadness', 0...",0.850919,0.584112,0.050878
733,1159941018764173317,"[('anger', 0.12963734567165375), ('sadness', 0...",0.907309,0.126229,0.006752,"[('anger', 0.16758503019809723), ('sadness', 0...",0.971827,0.039184,0.002246,"[('anger', 0.06847141683101654), ('sadness', 0...",0.899217,0.172037,0.012215


In [122]:
merged_df.rename(columns={"cosine_similarity_model_x": "cosine_similarity_model_text_base", "cosine_similarity_model_y": "cosine_similarity_model_image_base","kl_divergence_x" : "kl_divergence_text_base", "kl_divergence_y":"kl_divergence_image_base", "mean_squared_error_x": "mean_squared_error_text_base", "mean_squared_error_y":"mean_squared_error_image_base","predictions_x":"predictions_text_base", "predictions_y":"predictions_image_base"}, inplace=True)

In [124]:
merged_df = merged_df.merge(df3, on='conversation_id')

In [126]:
merged_df.rename(columns={"cosine_similarity_model": "cosine_similarity_model_multi_base","kl_divergence" : "kl_divergence_multi_base", "mean_squared_error": "mean_squared_error_multi_base","predictions":"predictions_multi_base"}, inplace=True)

In [138]:
merged_df

,conversation_id,predictions_text,cosine_similarity_model_text,kl_divergence_text,mean_squared_error_text,predictions_image,cosine_similarity_model_image,kl_divergence_image,mean_squared_error_image,predictions_multi,cosine_similarity_model_multi,kl_divergence_multi,mean_squared_error_multi
0,1096448460583260161,"[('anger', 0.15004052221775055), ('sadness', 0...",0.946335,0.071573,0.003591,"[('anger', 0.08058871328830719), ('sadness', 0...",0.698000,0.291042,0.023368,"[('anger', 0.06251640617847443), ('sadness', 0...",0.572058,0.465023,0.040149
1,1096541711847833601,"[('anger', 0.18943582475185394), ('sadness', 0...",0.661549,0.438556,0.029255,"[('anger', 0.08634641021490097), ('sadness', 0...",0.644016,0.405440,0.032207,"[('anger', 0.10931407660245895), ('sadness', 0...",0.769702,0.281330,0.020871
2,1095943507779305472,"[('anger', 0.20723669230937958), ('sadness', 0...",0.645878,0.560645,0.032595,"[('anger', 0.09861933439970016), ('sadness', 0...",0.874241,0.248578,0.013387,"[('anger', 0.10328623652458191), ('sadness', 0...",0.860980,0.262726,0.014989
3,1096757339749679104,"[('anger', 0.17028899490833282), ('sadness', 0...",0.903381,0.122912,0.006163,"[('anger', 0.10613662004470825), ('sadness', 0...",0.828795,0.211208,0.011112,"[('anger', 0.10246411710977554), ('sadness', 0...",0.841554,0.204232,0.010192
4,1100430895083868160,"[('anger', 0.19009549915790558), ('sadness', 0...",0.788205,0.318755,0.019081,"[('anger', 0.0764935314655304), ('sadness', 0....",0.386800,0.909694,0.056253,"[('anger', 0.0958344042301178), ('sadness', 0....",0.518807,0.703300,0.040995
...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1100873906594959360,"[('anger', 0.17107778787612915), ('sadness', 0...",0.794825,0.228380,0.013849,"[('anger', 0.1232147291302681), ('sadness', 0....",0.937599,0.066870,0.004488,"[('anger', 0.0943947434425354), ('sadness', 0....",0.889753,0.124676,0.007701
164,1098639572316160001,"[('anger', 0.13212314248085022), ('sadness', 0...",0.761314,0.319492,0.019725,"[('anger', 0.061768513172864914), ('sadness', ...",0.410181,0.802232,0.058960,"[('anger', 0.05107546225190163), ('sadness', 0...",0.364953,0.898039,0.068457
165,1100522521047252993,"[('anger', 0.1754831224679947), ('sadness', 0....",0.950110,0.064334,0.003066,"[('anger', 0.12668803334236145), ('sadness', 0...",0.953454,0.049281,0.002902,"[('anger', 0.07812617719173431), ('sadness', 0...",0.972629,0.037406,0.001757
166,1096408542297423872,"[('anger', 0.13744036853313446), ('sadness', 0...",0.878683,0.139792,0.008566,"[('anger', 0.07437614351511002), ('sadness', 0...",0.662158,0.329319,0.026832,"[('anger', 0.047963209450244904), ('sadness', ...",0.487106,0.588370,0.051766


In [140]:
merged_df = pd.read_csv("baseline_aug.csv")

In [141]:
metric_columns = [
    "cosine_similarity_model_text", "kl_divergence_text", "mean_squared_error_text",
    "cosine_similarity_model_image", "kl_divergence_image", "mean_squared_error_image",
    "cosine_similarity_model_multi", "kl_divergence_multi", "mean_squared_error_multi"
]

# Compute average values
averages = merged_df[metric_columns].mean().reset_index()

In [142]:
averages.columns = ['Metric', 'Average Value']

In [143]:
averages

,Metric,Average Value
0,cosine_similarity_model_text,0.728624
1,kl_divergence_text,0.515149
2,mean_squared_error_text,0.034614
3,cosine_similarity_model_image,0.817100
4,kl_divergence_image,0.358067
5,mean_squared_error_image,0.024206
6,cosine_similarity_model_multi,0.830667
7,kl_divergence_multi,0.327058
8,mean_squared_error_multi,0.021874


In [151]:
df = pd.read_csv("results_feb.csv")

In [145]:
df

,conversation_id,tweet_text,roberta-large-predictions,averaged_predictions,mapped_predictions,cosine_similarity_tweet_replies,clip_predictions,cosine_similarity_clip_replies,cosine_similarity_clip_tweet,similarity_bin,...,jsd_image_reply,tweet_entropy,reply_entropy,multi_entropy,KL_Divergence_tweet_distribution,KL_Divergence_clip_distribution,KL_Divergence_clip_tweet_dist,MSE_tweet_distribution,MSE_clip_distribution,MSE_clip_tweet_dist
0,1166008123263475712,#AlTenoorOverflow #Africafires#Angola#Zambia#T...,"[('anticipation', 0.18425847589969635), ('ange...","[('anger', 0.06943728003861536), ('fear', 0.48...","[('anger', 0.21514055782994462), ('fear', 0.15...",0.549003,"[('fear', 0.3789367), ('anger', 0.26606992), (...",0.805413,0.782031,0.6-0.8,...,0.372395,2.392732,1.894418,2.397760,0.659006,0.581619,0.479665,0.043325,0.035957,0.027494
1,1158024573243334657,Sage words from co-founder of the [USER] [USER...,"[('disgust', 0.9435292482376099), ('anger', 0....","[('anger', 0.12345011625812803), ('fear', 0.11...","[('anger', 0.3335972931162361), ('fear', 0.058...",0.741571,"[('surprise', 0.3979619), ('disgust', 0.192632...",0.718972,0.667360,0.6-0.8,...,0.410953,2.157165,2.474610,2.316720,0.340141,0.938059,0.299686,0.020341,0.028038,0.013925
2,1161462789795221504,So I know they are #ClimateChangedeniers.But I...,"[('disgust', 0.9702249765396118), ('anger', 0....","[('anger', 0.2677825772720873), ('fear', 0.104...","[('anger', 0.33216690109041486), ('fear', 0.12...",0.970299,"[('surprise', 0.39039624), ('anger', 0.3012097...",0.632252,0.620269,0.6-0.8,...,0.428990,2.045863,2.225498,2.320151,0.060455,0.407387,0.160361,0.002703,0.021938,0.008429
3,1163191441306398721,Is Climate Change real? That’s not up to us to...,"[('anticipation', 0.9576886892318726), ('optim...","[('anger', 0.15045814225567147), ('fear', 0.07...","[('anger', 0.08089625640880886), ('fear', 0.07...",0.938699,"[('anger', 0.45440218), ('disgust', 0.20690694...",0.622256,0.432749,0.4-0.6,...,0.458495,1.965242,2.367059,2.300349,0.113712,0.845224,0.113358,0.007616,0.030372,0.005374
4,1164233724772462592,Why isn’t everyone doing this?[URL] [USER] #cl...,"[('optimism', 0.792866587638855), ('sadness', ...","[('anger', 0.12882927371328437), ('fear', 0.17...","[('anger', 0.0580449479990004), ('fear', 0.028...",0.838172,"[('anger', 0.5921461), ('sadness', 0.21181808)...",0.565835,0.321836,0.2-0.4,...,0.516915,2.110238,2.389405,2.282167,0.305661,1.565800,0.283622,0.014076,0.058481,0.011233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730,1164909872409972736,"The diary of young ⁦[USER]⁩, 1994: never not d...","[('sadness', 0.9923811554908752), ('pessimism'...","[('anger', 0.029483021896698425), ('fear', 0.0...","[('anger', 0.011742866453751184), ('fear', 0.4...",0.501565,"[('fear', 0.418049), ('anger', 0.24365497), ('...",0.565783,0.583955,0.4-0.6,...,0.526592,1.593220,2.256675,2.139830,1.403162,0.933948,0.691342,0.054487,0.042847,0.034676
731,1157266962671689733,If terraforming is the transformation of anoth...,"[('fear', 0.5643811225891113), ('anticipation'...","[('anger', 0.08946653180516631), ('fear', 0.15...","[('anger', 0.06552404663067052), ('fear', 0.31...",0.636690,"[('anger', 0.5833039), ('sadness', 0.21278773)...",0.325762,0.357303,0.2-0.4,...,0.630028,2.188562,2.347650,2.308614,0.857563,1.886015,1.112878,0.028917,0.066930,0.032766
732,1162437587715289089,I’m with 700+ Catholic sisters at #LCWR2019. W...,"[('optimism', 0.9784810543060303), ('sadness',...","[('anger', 0.004858315480424416), ('fear', 0.0...","[('anger', 0.06499159918398285), ('fear', 0.12...",0.932122,"[('disgust', 0.36011955), ('anger', 0.34713572...",0.084196,0.287512,0.2-0.4,...,0.879451,2.052232,0.572612,2.439128,0.391660,3.720033,0.980267,0.031450,0.179255,0.084364
733,1159941018764173317,What an adventure! Three and a half years goes...,"[('joy', 0.9972921013832092), ('optimism', 0.8...","[('anger', 0.17794349822853445), ('fear', 0.03...","[('anger', 0.

In [154]:
df["MSE_tweet_distribution"].mean()

np.float64(0.04154851715382041)